In [88]:
import pandas as pd
import re
from lxml import etree
import shutil
import os
import math
import copy
import glob

In [89]:
#19-32
#113-128 chnaged the typos?
#491-504

In [90]:
def load_base_value_iden(file_path):
    if os.path.exists(file_path):
        
        # File exists, load the variable from the file
        if os.path.getsize(file_path) > 0:
                    # File is not empty, load the variable from the file
            with open(file_path, 'r') as file:
                
                variable_loaded = file.read().strip()
#                 print('loading')
#                 print(variable_loaded)
        else:
            # File exists but is empty, initialize with a default value
            variable_loaded = "11e1d6e4-74ba-49cc-9b55-1"
    else:
        # File does not exist, create the file and write the variable to it
#         print('+++++')
        print('creating file...')
        variable_to_store = "11e1d6e4-74ba-49cc-9b55-1"
        with open(file_path, 'w') as file:
            file.write(variable_to_store)
        variable_loaded = variable_to_store
        
    return variable_loaded
def save_base_value_iden(file_path,variable_to_store):
    
    # Write the variable to the file
    with open(file_path, 'w') as file:
        file.write(variable_to_store)
def increment_value_iden(value,file_path):

    str_last = value.split('-')[-1]
    last= int(str_last)
    last+=1
    
    exclude_last = value.split('-')[:-1]
    temp = '-'.join(exclude_last)

    new_identifier = temp + '-' + str(last)
    
    if os.path.exists(file_path):
        with open(file_path, 'w') as file:
            variable_to_store = new_identifier
#             print('writing')
#             print(variable_to_store)
            file.write(variable_to_store)
    else:
        print('load base value first')
def adjust_identifier(root_copy,filepath_base_identifier):
    base_identifier = load_base_value_iden(filepath_base_identifier)

    for index,e in enumerate(root_copy.iter()):
        if index == 10:#rsender identifier:
            e.text = base_identifier
            increment_value_iden(base_identifier,filepath_base_identifier)
def get_element_xpath(element):
    path = [element.tag.split('}')[-1]]
    for parent in element.iterancestors():
        parent_tag = parent.tag.split('}')[-1]  
        path.insert(0, parent_tag)
    return ' / '.join(path)
def read_xml_file(file_path_xml):
    result_xml_dict={}
    tree = etree.parse(file_path_xml)
    target_root = tree.getroot()
#     pattern = r'\{.*?\}'
    for element in target_root.iter():
        if element.attrib:
            xpath_base = get_element_xpath(element)
            for e,val in element.attrib.items():
                xpath = xpath_base + ' / ' + '@' + e
#                 print(f'-->{xpath}')
                result_xml_dict[xpath] = tree.getpath(element)
        else:
            xpath = get_element_xpath(element)
            result_xml_dict[xpath] = tree.getpath(element)
    return result_xml_dict
        
def get_element_full_path(tree,element):
    return tree.getpath(element)

def read_xml_file_2(file_path_xml):
    tree = etree.parse(file_path_xml)
    target_root = tree.getroot()
    for element in target_root.iter():
        xpath = tree.getpath(element)
        print(f"XPath: {xpath}")

def adjust_customisation_id(root_copy,value):
     for index,e in enumerate(root_copy.iter()):
        if index == 31:
            print("----")
            print("--customisation--")
            print("  ")
            print(f'{e} element text ->{e.text}')
            e.text = value
            print("  ")
            print("----")
            print("----")

            
        
def adjust_profile_id(root_copy,value):
    for index,e in enumerate(root_copy.iter()):
        if index == 33:
            print("----")
            print("--profile--")
            print("  ")
            print(f'{e} element text ->{e.text}')
            e.text = value
            print("  ")
            print("----")
            print("----")


def insert_for_attribute(target_element,insert_value,each_path):
    print('   ')
    print('------change attrib------')
    if len(target_element)>0:

        print(f'----<<--actual element value==>{target_element[0].attrib} type = {type(target_element[0].attrib)}')
        for key, value in target_element[0].attrib.items():
            print(f'val = {value}')
            target_element[0].attrib[key] = insert_value
        print(f'----<<--new attribute value==>{target_element[0].attrib}')

    else:
         print(f'--cannot get--<<--{each_path} | {target_element} is empty ')
    return target_element
def insert_for_element(target_element,insert_value,each_path):
    print('   ')
    print('------change element------')
    if len(target_element)>0:
        print(f'----<<--actual element text==>{target_element[0].text}')
        target_element[0].text = insert_value
        
        print(f'----<<--new element text==>{target_element[0].text}')
    else:
         print(f'---cannot get-<<--{each_path} | {target_element} is empty ')
    return target_element

def handle_descriptions(desc,index):#like a custom immigration gate,right before writng to file,remove if element
    if 'Field does not exist' in desc and index==0:
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        print('                                   ')
        
        print('RUMBLE')
        
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        print('                                   ')
    else:
        pass
#still there
#takes in desc and fieldone path.resolves into xpath
#remove element if still there at the end for filed does not exist
#change from 'nan' to '' if field exist with missingn value,adjust value_store first element as thats waht
#field description is referring to

def  delete_element_attrib(desc,target_root_copy,actual_xpath_absolute,r_h,each_path):
    attrib_yes = False
  
    if '@' in each_path:
        attrib_yes = True
        
    if 'Field does not exist' in desc:
        print(f'%%%%%%%%xpath find{actual_xpath_absolute} %%%%%%%%')
        print('                 ')
        print('Targeting element......')
        element_temp = target_root_copy.xpath(actual_xpath_absolute, namespaces=namespace_map)
        if len(element_temp)>0:
            element = element_temp[0]
            if attrib_yes:
                print(f'%%%%%%%% current attrib {element.attrib} %%%%%%%%')
                del element.attrib[key_attrib]
                print(f'%%%%%%%% new  attrib {element.attrib} %%%%%%%%')
#                 key_attrib = xpath_find.split('@')[-1]
#                 print(f'removing attrib {xpath_find}')
#                 print(f'%%%%%%%% current attrib {element.attrib} %%%%%%%%')
#                 del element.attrib[key_attrib]
#                 print(f'%%%%%%%% new  attrib {element.attrib} %%%%%%%%')
            else:
                print('%%%%%%%% Remove element %%%%%%%%')
                parent = element.getparent()

                if element.getparent() is not None:
                    element.getparent().remove(element)

                print('------------')
        else:
            print('xpath ')
        return target_root_copy
    else:
        print('%%%%%%%%%%%%%%%%')
        print('                  ')
        return target_root_copy


def extract_out_second_half(actual_xpath):#to be attachrd to path_base
    temp = actual_xpath.split('/')[-1]
    return temp
def handle_base_path(s):
    start = r_h[s]
#     print(f'start {start}')
    a= start
    specific_elem = a.split('/')[-1]
#     print(f'specific elem{specific_elem}')
    temp = a.split('/')[:-1]
#     print('temp',temp)
    temp_a = '/'.join(temp)
#     print('temp_a',temp_a)

    temp_b = temp[-1].split('[')[0]
    temp_c = temp
    temp_c[-1] = temp_b
    temp_d = '/'.join(temp_c)
#     print('temp_d',temp_d)
#     print('   ')
    return temp_d
            
def evaluate_if_got_multiple(current_test_case):
    occur_again = False
    for index,fields in enumerate(current_test_case):
        xpath_find = fields[0].strip()
        if index == 0:
            field_one = xpath_find
        if (xpath_find == field_one) and index != 0:
            occur_again = True
            return True
    return occur_again
def prepare_data_for_multiple(current_test_case):
    holding = []
    values_store=[]
    for index,fields in enumerate(current_test_case):
        
        path_to_store = fields[0]
        value = fields[1]
        values_store.append(value)
        if index == 0:
            temp = []
            
            field_one = path_to_store
            temp.append(path_to_store)
            
        elif (path_to_store == field_one) and (index != 0):
            holding.append(temp)
            temp = []

        
            temp.append(path_to_store)
        else:
#             print('  ----  ')
#             print('    ')
#             print(temp)
#             print('  ----  ')
#             print('    ')
            temp.append(path_to_store)
    holding.append(temp)
    return holding , values_store

def fix_space_typo(temp):
    temp_2 = temp.split("/")
    for index,i in enumerate(temp_2):
        temp_2[index] = i.strip()
    temp_3 = ' / '.join(temp_2)
    return temp_3

In [91]:
output_folder = '/Users/cheongray/iras_api_clean_1/4_format_validation'
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

output_file = os.path.join(output_folder, 'df_full.csv')
output_file_original = os.path.join(output_folder, 'df_full_back_up.csv')
if os.path.exists(output_file):
    os.remove(output_file)
if os.path.exists(output_file_original):
    os.remove(output_file_original)


In [92]:
output_folder = '/Users/cheongray/iras_api_clean_1/4_format_validation'
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

output_file = os.path.join(output_folder, 'df_full.csv')
output_file_original = os.path.join(output_folder, 'df_full_back_up.csv')
if os.path.exists(output_file):
    os.remove(output_file)
if os.path.exists(output_file_original):
    os.remove(output_file_original)

file_path = r"/Users/cheongray/iras_api_pytest_1/AP_SP_files/AP_SP_FINAL_21.xlsx"
sheet_name = '4_Format Validation'

path_to_sample_xml_files_folder = r"/Users/cheongray/iras_api_clean_1/sample_xml_files"


df_s = pd.read_excel(file_path, sheet_name=sheet_name)
df_s_temp = df_s.copy()
df_s_temp['Test Case ID'] = df_s_temp.index.astype(float)#REMINDER,now test_case_id is 1_1
# df_s_peppol = df_s_temp[(df_s_temp['Standard'] == 'PEPPOL') & (df_s_temp['Document Type'] == 'Invoice')].copy()
# df_s_peppol['Test Case ID'] = df_s_peppol['Test Case ID'] + 1
# df_s_peppol.reset_index(drop=True)
df_s_temp['Test Case ID'] = df_s_temp['Test Case ID'] + 1
df_s_temp.to_csv(output_file, index=False)
df_s_temp.to_csv(output_file_original, index=False)

In [93]:

df_s_temp_subset = df_s_temp.copy()

unique_descriptions = set()

target_elements_syn = []

for index,row in df_s_temp_subset.iterrows():
    filled_columns = row.dropna().index
    test_case_id = f' {index}'
    descript = row['Test Case Description']
#     print(f'Test case {index+1}')
#     print(f'Test case description : {descript}')
    unique_descriptions.add(descript)#just to build function_mapping
    temp={}
    temp['Rule ID'] = row['Rule ID']
    temp['Standard'] = row['Standard']
    temp['Document Type'] = row['Document Type']
    temp['BIS/PINT'] = row['BIS/PINT']
    temp['Test Case ID'] = int(row['Test Case ID'])
    temp['descript'] = descript
    temp['Expected Result'] = row['Expected Result Response (for 400 response)']
    temp['Status'] = row['Expected Result']
    temp['Fields'] = []
    temp['Sample'] = row['Sample']
    for i, col in enumerate(filled_columns, start=1):
        field_name = f'Field {i}'
        
        if field_name in df_s_temp_subset.columns:
            # Process the data as needed
#             print(f'Column: {field_name}, Value: {row[field_name]}')
   
            
            temp_2 = [] #each field will be one array,can do dict but mor steps when want access
            if not pd.isna(row[field_name]):
                
                field_value = field_name + ' Value'
                temp_2.append(fix_space_typo(row[field_name]))
                temp_2.append(row[field_value])
                temp['Fields'].append(temp_2)
        
    target_elements_syn.append(temp)

            


In [94]:

#gets the needed files for 2_Conditional_Input_Validation
file_paths_xml_files = {}

for i,test_case in enumerate(target_elements_syn):
    
    sample_to_use = test_case['Sample']
    
    num = sample_to_use.split(' ')[-1]
    file_name = 'sample' + '_' + num + '.xml'
    
    path_to_sample_xml_files = path_to_sample_xml_files_folder + '/' + file_name
    
    file_paths_xml_files[sample_to_use] = path_to_sample_xml_files


In [95]:
file_paths_xml_files

{'Sample 1': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml',
 'Sample 3': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_3.xml',
 'Sample 2': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml',
 'Sample 4': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_4.xml',
 'Sample 11': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_11.xml',
 'Sample 12': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_12.xml',
 'Sample 30': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_30.xml',
 'Sample 13': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_13.xml',
 'Sample 31': '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_31.xml'}

In [96]:
folder_name

'/Users/cheongray/iras_api_make_xml/store_manual_fix_files'

In [97]:
folder_name = output_folder + '/xml_files_for'+'_'+sheet_name

filepath_base_identifier = '/Users/cheongray/iras_api_make_xml/identifier_store/stored_base_identifier_conditional_validation.txt'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
if os.path.exists(folder_name):
    shutil.rmtree(folder_name)
    os.makedirs(folder_name)
function_mapping = {
    'Field exists with missing value':0,
    'Field does not exist': 1
}
namespace_map = {
    'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
    'xs': 'http://www.w3.org/2001/XMLSchema',
    'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
    'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
    }

In [98]:

# # save_base_value_iden(filepath_base_identifier,'11e1d6e4-74ba-49cc-9b55-1')







# def handle_multiple(current_test_case,test_case_id,r_h,target_root,desc):
#     print(f' {test_case_id}  ')
#     whole_count_pos = 0
#     hold_1 , value_store = prepare_data_for_multiple(current_test_case)
#     print('------------')
#     print(hold_1)
#     print('   ')
#     field_1 = hold_1[0][0]
#     base_path_field_1 = handle_base_path(field_1)

#     print('   ')
#     value_store_key = 0
#     temp_base_path = base_path_field_1
#     target_root_copy = copy.deepcopy(target_root)
#     for index,each_case in enumerate(hold_1):
        
        

#         path_base = temp_base_path + f'[{index+1}]' + '/'
        

#         for each_path in each_case:
            
#             print('+++++++++++++++++++++++++')
            

            
#             insert_value = str(value_store[value_store_key])
#             print('+++++++++++++++++++++++++')
#             print('---------------')
#             print('   ')
            
#             print(f'insert value = {value_store[value_store_key]}')
#             if 'Field exists with missing value' in desc and value_store_key == 0:
                
#                 print(f'old insert value {insert_value} type is {type(insert_value)} id is {value_store_key}')
#                 print(f'##Description = {desc}##')
#                 insert_value = ''
#                 print(f'new insert value {insert_value}')
#             value_store_key +=1
        
#             print(f'value_store_key === > {value_store_key}')
#             print(' ')
#             base_path_each_path = handle_base_path(each_path)

#             if base_path_each_path == base_path_field_1:

#                 print('   ')
#                 print(f'----<<--Ammend base path for {each_path} ->>-----')
# #                 if each_path == field_1 and whole_count_pos==0:
# #                     print(']]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]')
# #                     print(f'each_path {each_path} ')
# #                     target_root_copy = delete_element_attrib(desc,target_root_copy,each_path,r_h)
# #                 print('   ')
#                 if each_path in r_h:
#                     actual_xpath = r_h[each_path]
#                     temp = extract_out_second_half(actual_xpath)
#                     actual_xpath_absolute = path_base + temp
#                     print(f'----<<--actual amended base path ==>{actual_xpath_absolute}')
#                     target_element = target_root_copy.xpath(actual_xpath_absolute,namespaces = namespace_map)

#                     print(f'----<<--actual path {each_path} index == {whole_count_pos}')
#                     if '@' in each_path:
#                         print(f'target element ==>{target_element}')
#                         print(f'>>>>>>target_element.attrib = {target_element[0].attrib}')
#                         print(f'>>>>>>target_element.text = {target_element[0].text}')
#                         target_element = insert_for_attribute(target_element,insert_value,each_path)
#                         print('             ')
                        
#                         print('   ')
#                     else:
#                         target_element = insert_for_element(target_element,insert_value,each_path)
#                     if whole_count_pos == 0:
#                         print(']]]]]]]]]]]]]]]]]base_path_each_path == base_path_field_1]]]]]]]]]]]]]]]]]]')
#                         print(f'each path = {each_path}')
#                         print(f'{actual_xpath_absolute}')
#                         target_root_copy = delete_element_attrib(desc,target_root_copy,actual_xpath_absolute,r_h,each_path)
#                         print('<<<<last>>>')
      

#             else:#base_path_each_path != base_path_field_1:
#                     print('   ')
#                     print(f'----<<--just change straight away {each_path} index == {whole_count_pos}->>')
#                     print('   ')
#                     if each_path in r_h:
#                         actual_xpath = r_h[each_path]
#                     target_element = target_root_copy.xpath(actual_xpath,namespaces = namespace_map)
#                     print('    ')
#                     print('===============')
#                     print(f'acutal path : {actual_xpath}')
                    
                    
#                     print('   ')
#                     if '@' in each_path:
#                         print('   ')
#                         print(f'target element {target_element}')
#                         print('------change attrib------')
#                         target_element = insert_for_attribute(target_element,insert_value,each_path)
                       
#                         print('   ')
#                     else:
#                         print('   ')
#                         print(f'target element {target_element}')
#                         print('------change element------')
#                         target_element = insert_for_element(target_element,insert_value,each_path)
                        

#                         print('   ')
#                     if whole_count_pos == 0:
#                         print(']]]]]]]]]]]]]]]base_path_each_path != base_path_field_1]]]]]]]]]]]]]]]]]]]]')
#                         print(f'each path = {each_path}')
#                         print(f'{actual_xpath_absolute}')
#                         target_root_copy = delete_element_attrib(desc,target_root_copy,actual_xpath,r_h)
#                         print('<<<<last>>>')
# #             if whole_count_pos == 0:

# #                 print(f'{each_path}')
# #                 print(f'{ea}')
# #                 if '@' in each_path:
# #                     pass
# #                 print('<<<<last>>>')
#             whole_count_pos += 1
  
    

#     print('   ')
#     print('             ')
#     print(value_store_key)
#     print('             ')
#     print(value_store)
#     return target_root_copy


# #[['ubl:Invoice / cac:TaxTotal / cbc:TaxAmount', 'ubl:Invoice / cac:TaxTotal / cbc:TaxAmount / @currencyID']
# #, ['ubl:Invoice / cac:TaxTotal / cbc:TaxAmount', 'ubl:Invoice / cac:TaxTotal / cbc:TaxAmount / @currencyID', 'ubl:Invoice / cbc:DocumentCurrencyCode', 'ubl:Invoice / cbc:TaxCurrencyCode']]   
    






    
    
    



# KEY_NOT_FOUND=[]
# elements_to_remove = []
# error_prone_multiple = []
# for i,test_case in enumerate(target_elements_syn):
    
# #     current_test_case = test_case['Fields']
# #     test_case_id = test_case['Test Case ID']
# #     desc = test_case['descript']
# #     sample_to_use = test_case['Sample']
# #     rule_id = test_case['Rule ID']
        
#     current_test_case = test_case['Fields']
#     test_case_id = test_case['Test Case ID']

#     print('############=================############')
#     print(f'TEST CAES ID {test_case_id}')
    
#     print('############=================############')

#     desc = test_case['descript']
#     sample_to_use = test_case['Sample']
#     standard  = test_case['Standard']
#     biz_pint = test_case['BIS/PINT']
#     rule_id = test_case['Rule ID']
    
    
#     f_p = file_paths_xml_files[sample_to_use]
    
#     print(f'using {f_p}')
#     print('   ')
#     r_h = proper_mapping_final(f_p)
    
#     if not r_h:
#         print(f'cannot make r_h : {test_case_id}  sample :{sample_to_use}')
#         continue
#     tree = etree.parse(f_p)
#     target_root = tree.getroot()
# #     print(f'test case id :{test_case_id}')
# #     print(desc)
#     original_tree_copy = copy.deepcopy(target_root)
#     if standard == 'PEPPOL':
#         if biz_pint == 'BIS':
#             value = 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0'
#             adjust_profile_id(original_tree_copy,value)
#             value_cus = 'urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0'
#             adjust_customisation_id(original_tree_copy,value_cus)

#         if biz_pint == 'PINT':
#             value = 'urn:peppol:bis:billing'
#             adjust_profile_id(original_tree_copy,value)
#             value_cus = 'urn:peppol:pint:billing-1@sg-1'
#             adjust_customisation_id(original_tree_copy,value_cus)


#     if standard == 'Non-PEPPOL':
#         if biz_pint == 'BIS':
#             value = 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0'
#             adjust_profile_id(original_tree_copy,value)
#             value_cus = 'urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:LocalTaxInvoice:sg:1.0'
#             adjust_customisation_id(original_tree_copy,value_cus)

#         if biz_pint == 'PINT':
#             value = 'urn:peppol:bis:billing'
#             adjust_profile_id(original_tree_copy,value)
#             value_cus = 'urn:peppol:pint:billing-1@sg-1:LocalTaxInvoice:sg:1.0'
#             adjust_customisation_id(original_tree_copy,value_cus)
            
            
            
            
#     adjust_identifier(original_tree_copy,filepath_base_identifier)
#     lenght_of_fields = len(current_test_case)
#     occur_again = evaluate_if_got_multiple(current_test_case)
#     if not occur_again:
#         print('CURRENT TEST CASE')
#         print(current_test_case)
#         for index,fields in enumerate(current_test_case):

#             attrib_yes = False
#             xpath_find = fields[0].strip()
            
#             field_value = str(fields[1])
#             print('++++++++++++++++++')
#             print('++++++++bododododo++++++++++')
#             if pd.isna(fields[1]) and 'does not exist' not in desc and 'missing value' not in desc:
#                 print(f'hulala back boi')
#                 field_value = 'NA'
#                 print(field_value)
       
#             if '@' in xpath_find:
#                     attrib_yes = True
            
#             if 'does not exist' in desc and index == 0:
#                 if '@' in xpath_find:
#                     print('why look at u youre an attribute')
#                 remove_element_flag = True
#                 print(f'xpath_find{xpath_find} |||||||')
#                 print('    ')
#                 temp = []
#                 if xpath_find in r_h:
#                     xpath_target = r_h[xpath_find]
#                 else:
#                     print(f'while trying to find xpath targtet to store in elements_to_remove,could not find key {xpath_find}')
#                 elements_to_remove_file_name = rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
#                 temp.append(elements_to_remove_file_name)
#                 temp.append(xpath_find)
#                 temp.append(xpath_target)
#                 temp.append(f_p)
#                 elements_to_remove.append(temp)
#             if 'missing value' in desc and index == 0:
#                 print(f'old insert value {field_value} type is {type(field_value)}')
#                 print('      ')
#                 print(f'id == {index}')
#                 print(f'##Description = {desc}##')
#                 field_value = ''
#                 print(f'new insert value {field_value}')   
#             if xpath_find in r_h:
#                 xpath_target = r_h[xpath_find]
#                 if original_tree_copy.xpath(xpath_target, namespaces=namespace_map):
#                     element = original_tree_copy.xpath(xpath_target, namespaces=namespace_map)[0]
#                     if not attrib_yes:
#                         print(f'-non multiple-->insert test case id{test_case_id}  fieled value-->{field_value} xpath=={xpath_target}')    
#                         element.text = field_value
#                     else:
#                         key_attrib = xpath_find.split('@')[-1]
#                         print(f'test case id{test_case_id}  key attrib -->{key_attrib}')
#                         element.attrib[key_attrib] = field_value
#                         print('                                           ')
#                 if 'Field does not exist' in desc and index == 0:

#                         print('Targeting element......')
#                         if xpath_find in r_h:
#                             xpath_target = r_h[xpath_find]
#                             element_temp = original_tree_copy.xpath(xpath_target, namespaces=namespace_map)
#                             if len(element_temp)>0:
#                                 element = element_temp[0]
#                                 if attrib_yes:
#                                     del element.attrib[key_attrib]
#                                 else:

#                                     parent = element.getparent()

#                                     if element.getparent() is not None:
#                                         element.getparent().remove(element)
#                             else:
#                                 print(' ')
#                                 print(f'element to be removed {element} does not exist')
#             else:#+++
#                 temp=[]
#                 temp.append(f'test case:{test_case_id} | uses {sample_to_use} key not found for {xpath_find}')
#                 print(f'--->test case:{test_case_id} | uses {sample_to_use} key not found for {xpath_find}')
#                 KEY_NOT_FOUND.append(temp)
#         target_root = original_tree_copy
#     else:
#         error_prone_multiple.append(test_case_id)
#         target_root = handle_multiple(current_test_case,test_case_id,r_h,original_tree_copy,desc)
#     tree = etree.ElementTree(target_root)
#     indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
#     filepath = os.path.join(folder_name, indexed_filename)
#     print(f'writing {filepath}')
#     tree.write(filepath, encoding='utf-8', xml_declaration=True)
             
       
        

In [99]:
# def process_xml_files_in_folder_Invoice_ID(folder_path,elements_to_remove):
#     check_how_many_removed = []
#     remove_from_root = []
#     xml_files = glob.glob(os.path.join(folder_path, '*.xml'))
#     for index_file,xml_file in enumerate(xml_files):
#         name = xml_file.split('/')[-1]
#         for e in elements_to_remove:
            
#             if name in e:
#                 file_name = e[0]
# #                 xpath_target = e[-2]
#                 xpath_find = e[1]
#                 f_p = e[-1]
#                 tree_temp = etree.parse(f_p)
#                 r_h = proper_mapping_final(f_p)
#                 if xpath_find in r_h:
#                     xpath_target = r_h[xpath_find]
#                     target_root = tree_temp.getroot()
#                     element = target_root.xpath(xpath_target,namespaces=namespace_map)[0]
#                     print(f'file name {file_name}')
#                     print(f'element text {element.text}')
#                     if element.getparent() is not None:
#                         print('++++++')
#                         element.getparent().remove(element)
#                         print(element)
#                         check_how_many_removed.append(xml_file)
#                         print('     ')

#                     else:
#                         remove_from_root.append(xpath_find)
#                         target_root.remove(element)
#                     tree = etree.ElementTree(target_root)
#                     tree.write(xml_file, encoding='utf-8', xml_declaration=True)
#                     continue
#             else:
#                 continue
                
#     return check_how_many_removed

# check_num = process_xml_files_in_folder_Invoice_ID(folder_name,elements_to_remove)

In [100]:
# to_skip_for_manual_fix = []
# for i in check_num:
#     a= i.split('Test_case')
#     to_skip_for_manual_fix.append(int(a[1].split('_')[1]))
# total_cases = [i for i in range(1,769)]
# to_manual_fix = [item for item in total_cases if item not in to_skip_for_manual_fix]

In [101]:
# filefolderpath = folder_name
# c=0
# xml_files = glob.glob(os.path.join(filefolderpath, '*.xml'))
# for xml in xml_files:
#     c+=1
#     print(xml)

In [102]:
# get_mapping_for_xml_credit('/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml')

In [103]:
#bob the builder

In [104]:
# folder_name = output_folder + '/try_2_fix_xml_files_for'+'_'+sheet_name

In [115]:
def load_base_value_iden(file_path):
    if os.path.exists(file_path):
        
        # File exists, load the variable from the file
        if os.path.getsize(file_path) > 0:
                    # File is not empty, load the variable from the file
            with open(file_path, 'r') as file:
                
                variable_loaded = file.read().strip()
#                 print('loading')
#                 print(variable_loaded)
        else:
            # File exists but is empty, initialize with a default value
            variable_loaded = "11e1d6e4-74ba-49cc-9b55-1"
    else:
        # File does not exist, create the file and write the variable to it
#         print('+++++')
        print('creating file...')
        variable_to_store = "11e1d6e4-74ba-49cc-9b55-1"
        with open(file_path, 'w') as file:
            file.write(variable_to_store)
        variable_loaded = variable_to_store
        
    return variable_loaded
def save_base_value_iden(file_path,variable_to_store):
    
    # Write the variable to the file
    with open(file_path, 'w') as file:
        file.write(variable_to_store)
def increment_value_iden(value,file_path):

    str_last = value.split('-')[-1]
    last= int(str_last)
    last+=1
    
    exclude_last = value.split('-')[:-1]
    temp = '-'.join(exclude_last)

    new_identifier = temp + '-' + str(last)
    
    if os.path.exists(file_path):
        with open(file_path, 'w') as file:
            variable_to_store = new_identifier
#             print('writing')
#             print(variable_to_store)
            file.write(variable_to_store)
    else:
        print('load base value first')
def adjust_identifier(root_copy,filepath_base_identifier):
    base_identifier = load_base_value_iden(filepath_base_identifier)

    for index,e in enumerate(root_copy.iter()):
        if index == 10:#rsender identifier:
            e.text = base_identifier
            increment_value_iden(base_identifier,filepath_base_identifier)
    return root_copy
def get_element_xpath(element):
    path = [element.tag.split('}')[-1]]
    for parent in element.iterancestors():
        parent_tag = parent.tag.split('}')[-1]  
        path.insert(0, parent_tag)
    return ' / '.join(path)
def read_xml_file(file_path_xml):
    result_xml_dict={}
    tree = etree.parse(file_path_xml)
    target_root = tree.getroot()
#     pattern = r'\{.*?\}'
    for element in target_root.iter():
        if element.attrib:
            xpath_base = get_element_xpath(element)
            for e,val in element.attrib.items():
                xpath = xpath_base + ' / ' + '@' + e
#                 print(f'-->{xpath}')
                result_xml_dict[xpath] = tree.getpath(element)
        else:
            xpath = get_element_xpath(element)
            result_xml_dict[xpath] = tree.getpath(element)
    return result_xml_dict
        
def get_element_full_path(tree,element):
    return tree.getpath(element)

def read_xml_file_2(file_path_xml):
    tree = etree.parse(file_path_xml)
    target_root = tree.getroot()
    for element in target_root.iter():
        xpath = tree.getpath(element)
        print(f"XPath: {xpath}")
def check_xpath_exists(invoice_dict,xpath):
    if xpath in invoice_dict:
        return True
    else:
        return False
def insert_invoice_dict(invoice_dict_xpath,insert_value,xpath_exists):
    if not xpath_exists:
        invoice_dict[xpath] = insert_value
    else:
        invoice_dict[xpath].append(inse)
    

def get_mapping_for_xml_invoice(file_path_of_sample_xml):

    namespace_map = {
        'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
        'xs': 'http://www.w3.org/2001/XMLSchema',
        'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
        'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
        'ubl': 'urn:oasis:names:specification:ubl:schema:xsd:Invoice-2',
        'xsi': "http://www.w3.org/2001/XMLSchema-instance",
        'xsd':"http://www.w3.org/2001/XMLSchema",
        'udt':"urn:un:unece:uncefact:data:specification:UnqualifiedDataTypesSchemaModule:2" ,
        'qdt':"urn:oasis:names:specification:ubl:schema:xsd:QualifiedDatatypes-2",
        'ext':"urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2",
        'ccts':"urn:un:unece:uncefact:documentation:2"
    #     'xmlns' : "urn:oasis:names:specification:ubl:schema:xsd:Invoice-2"
    }
    tree = etree.parse(file_path_of_sample_xml)
    target_root = tree.getroot()
    # format_key_map = []
    invoice_dict={}
    for element in target_root.iter():
        if isinstance(element, etree._Comment):
            continue
        ancestors = [e.tag for e in element.iterancestors()][::-1] + [element.tag]

        string_ancestors = [e if isinstance(e, str) else str(e) for e in ancestors]

        path = ' / '.join(string_ancestors)
        for prefix, uri in namespace_map.items():
            path = path.replace('{' + uri + '}', prefix + ':')
        if 'sb:StandardBusinessDocument / ubl:Invoice / ' in path:
            path = path.replace('sb:StandardBusinessDocument / ', '')
            if element.attrib:
                xpath_base = path
#                 print(f'==={1}======')
#                 print(element)
#                 print(element.text)
#                 print(xpath_base)
#                 print('============')
#                 print('           ')
                xpath_exists = check_xpath_exists(invoice_dict,xpath_base)
                if not xpath_exists:
                    invoice_dict[xpath_base] = [tree.getpath(element)]
                    
                else:
                    invoice_dict[xpath_base].append(tree.getpath(element))
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e
                    if xpath in invoice_dict:
                         invoice_dict[xpath].append(tree.getpath(element))
                    else:
                        invoice_dict[xpath] = [tree.getpath(element)]
            else:                
#                 print(f'={909}=={1}======')
#                 print(element)
                
#                 print(element.text)
#                 print(xpath_base)
#                 print(path)
#                 print('============')
#                 print('           ')
                insert_value = tree.getpath(element)
                xpath_exists = check_xpath_exists(invoice_dict,path)
                if not xpath_exists:
                    invoice_dict[path] = [insert_value]
                else:
                    invoice_dict[path].append(insert_value)

        else:
            if element.attrib:
#                 print(f'==={2}======')
#                 print(element)
#                 print(path)
#                 print('============')
#                 print('           ')
                xpath_base = get_element_xpath(element)
                invoice_dict[xpath_base] = tree.getpath(element)
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e
    #                 print(f'-->{xpath}')


                    invoice_dict[xpath] = [tree.getpath(element)]
            else:
#                 print(f'==={2}======')
#                 print(element)
#                 print(path)
#                 print('============')
#                 print('           ')
                xpath = get_element_xpath(element)#defined on top
                

                
                insert_value = tree.getpath(element)
#                 print('=========')
#                 print(element)
#                 print(xpath)
#                 print(f'insert val = {insert_value}')
#                 print('============')
#                 print('           ')
                xpath_exists = check_xpath_exists(invoice_dict,xpath)
                if not xpath_exists:
                    invoice_dict[xpath] = [insert_value]
                else:
                    
                    invoice_dict[xpath].append(insert_value)
    r_h = invoice_dict
    return r_h
def get_mapping_for_xml_credit(file_path_of_sample_xml):

    namespace_map = {
        'sb': 'http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader',
        'xs': 'http://www.w3.org/2001/XMLSchema',
        'cac': 'urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2',
        'cbc': 'urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2',
        'ubl': 'urn:oasis:names:specification:ubl:schema:xsd:CreditNote-2',
        'xsi': "http://www.w3.org/2001/XMLSchema-instance",
        'xsd':"http://www.w3.org/2001/XMLSchema",
        'udt':"urn:un:unece:uncefact:data:specification:UnqualifiedDataTypesSchemaModule:2" ,
        'qdt':"urn:oasis:names:specification:ubl:schema:xsd:QualifiedDatatypes-2",
        'ext':"urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2",
        'ccts':"urn:un:unece:uncefact:documentation:2" 
    }
    tree = etree.parse(file_path_of_sample_xml)
    target_root = tree.getroot()
    # format_key_map = []
    invoice_dict={}
    for element in target_root.iter():
        if isinstance(element, etree._Comment):
            continue
        ancestors = [e.tag for e in element.iterancestors()][::-1] + [element.tag]

        string_ancestors = [e if isinstance(e, str) else str(e) for e in ancestors]

        path = ' / '.join(string_ancestors)
        for prefix, uri in namespace_map.items():
            path = path.replace('{' + uri + '}', prefix + ':')
        if 'sb:StandardBusinessDocument / ubl:CreditNote / ' in path:

            path = path.replace('sb:StandardBusinessDocument / ', '')
            if element.attrib:
                xpath_base = path
#                 print(f'==={1}======')
#                 print(element)
#                 print(element.text)
#                 print(xpath_base)
#                 print('============')
#                 print('           ')
                xpath_exists = check_xpath_exists(invoice_dict,xpath_base)
                if not xpath_exists:
                    invoice_dict[xpath_base] = [tree.getpath(element)]
                    
                else:
                    invoice_dict[xpath_base].append(tree.getpath(element))
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e
                    if xpath in invoice_dict:
                         invoice_dict[xpath].append(tree.getpath(element))
                    else:
                        invoice_dict[xpath] = [tree.getpath(element)]
            else:                
#                 print(f'={909}=={1}======')
#                 print(element)
                
#                 print(element.text)
#                 print(xpath_base)
#                 print(path)
#                 print('============')
#                 print('           ')
                insert_value = tree.getpath(element)
                xpath_exists = check_xpath_exists(invoice_dict,path)
                if not xpath_exists:
                    invoice_dict[path] = [insert_value]
                else:
                    invoice_dict[path].append(insert_value)

        else:
            if element.attrib:
#                 print(f'==={2}======')
#                 print(element)
#                 print(path)
#                 print('============')
#                 print('           ')
                xpath_base = get_element_xpath(element)
                invoice_dict[xpath_base] = tree.getpath(element)
                for e,val in element.attrib.items():
                    xpath = xpath_base + ' / ' + '@' + e
    #                 print(f'-->{xpath}')


                    invoice_dict[xpath] = [tree.getpath(element)]
            else:
#                 print(f'==={2}======')
#                 print(element)
#                 print(path)
#                 print('============')
#                 print('           ')
                xpath = get_element_xpath(element)#defined on top
                

                
                insert_value = tree.getpath(element)
#                 print('=========')
#                 print(element)
#                 print(xpath)
#                 print(f'insert val = {insert_value}')
#                 print('============')
#                 print('           ')
                xpath_exists = check_xpath_exists(invoice_dict,xpath)
                if not xpath_exists:
                    invoice_dict[xpath] = [insert_value]
                else:
                    
                    invoice_dict[xpath].append(insert_value)
    r_h = invoice_dict
    return r_h






def credit_or_invoice(file_path):
#     print('********')
#     print(file_path)
    tree = etree.parse(file_path)
    target_root = tree.getroot()
    for index,e in enumerate(target_root.iter()):
        if index ==11:
            if 'Credit' in e.text:
                return 'credit'
            elif 'Invoice' in e.text:
                return 'invoice'
            else:
                return 'undefined'
def proper_mapping_final(file_path):

    if credit_or_invoice(file_path) == 'credit':
        r_h = get_mapping_for_xml_credit(file_path)
        return r_h
    elif credit_or_invoice(file_path) == 'invoice':
        r_h = get_mapping_for_xml_invoice(file_path)
        return r_h
    else:
        return None
def get_element_xpath(element):
    path = [element.tag.split('}')[-1]]
    for parent in element.iterancestors():
        parent_tag = parent.tag.split('}')[-1]  
        path.insert(0, parent_tag)
    return ' / '.join(path)
def credit_or_invoice(file_path):
#     print('********')
#     print(file_path)
    tree = etree.parse(file_path)
    target_root = tree.getroot()
    for index,e in enumerate(target_root.iter()):
        if index ==11:
            if 'Credit' in e.text:
                return 'credit'
            elif 'Invoice' in e.text:
                return 'invoice'
            else:
                return 'undefined'
def proper_mapping_final(file_path):

    if credit_or_invoice(file_path) == 'credit':
        r_h = get_mapping_for_xml_credit(file_path)
        return r_h
    elif credit_or_invoice(file_path) == 'invoice':
        r_h = get_mapping_for_xml_invoice(file_path)
        return r_h
    else:
        return None
def adjust_customisation_id(root_copy,value):
     for index,e in enumerate(root_copy.iter()):
        if index == 31:
            print("----")
            print("--customisation--")
            print("  ")
            print(f'{e} element text ->{e.text}')
            e.text = value
            print("  ")
            print("----")
            print("----")

            
        
def adjust_profile_id(root_copy,value):
    for index,e in enumerate(root_copy.iter()):
        if index == 33:
            print("----")
            print("--profile--")
            print("  ")
            print(f'{e} element text ->{e.text}')
            e.text = value
            print("  ")
            print("----")
            print("----")


def insert_for_attribute(target_element,insert_value,each_path):
    print('   ')
    print('------change attrib------')
    if len(target_element)>0:

        print(f'----<<--actual element value==>{target_element[0].attrib} type = {type(target_element[0].attrib)}')
        for key, value in target_element[0].attrib.items():
            print(f'val = {value}')
            target_element[0].attrib[key] = insert_value
        print(f'----<<--new attribute value==>{target_element[0].attrib}')

    else:
         print(f'--cannot get--<<--{each_path} | {target_element} is empty ')
    return target_element
def insert_for_element(target_element,insert_value,each_path):
    print('   ')
    print('------change element------')
    if len(target_element)>0:
        print(f'----<<--actual element text==>{target_element[0].text}')
        target_element[0].text = insert_value
        
        print(f'----<<--new element text==>{target_element[0].text}')
    else:
         print(f'---cannot get-<<--{each_path} | {target_element} is empty ')
    return target_element

def handle_descriptions(desc,index):#like a custom immigration gate,right before writng to file,remove if element
    if 'Field does not exist' in desc and index==0:
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        print('                                   ')
        
        print('RUMBLE')
        
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        print('                                   ')
    else:
        pass
#still there
#takes in desc and fieldone path.resolves into xpath
#remove element if still there at the end for filed does not exist
#change from 'nan' to '' if field exist with missingn value,adjust value_store first element as thats waht
#field description is referring to

def  delete_element_attrib(desc,target_root_copy,actual_xpath_absolute,r_h,each_path):
    attrib_yes = False
  
    if '@' in each_path:
        attrib_yes = True
        
    if 'Field does not exist' in desc:
        print(f'%%%%%%%%xpath find{actual_xpath_absolute} %%%%%%%%')
        print('                 ')
        print('Targeting element......')
        element_temp = target_root_copy.xpath(actual_xpath_absolute, namespaces=namespace_map)
        if len(element_temp)>0:
            element = element_temp[0]
            if attrib_yes:
                print(f'%%%%%%%% current attrib {element.attrib} %%%%%%%%')
                del element.attrib[key_attrib]
                print(f'%%%%%%%% new  attrib {element.attrib} %%%%%%%%')
#                 key_attrib = xpath_find.split('@')[-1]
#                 print(f'removing attrib {xpath_find}')
#                 print(f'%%%%%%%% current attrib {element.attrib} %%%%%%%%')
#                 del element.attrib[key_attrib]
#                 print(f'%%%%%%%% new  attrib {element.attrib} %%%%%%%%')
            else:
                print('%%%%%%%% Remove element %%%%%%%%')
                parent = element.getparent()

                if element.getparent() is not None:
                    element.getparent().remove(element)

                print('------------')
        else:
            print('xpath ')
        return target_root_copy
    else:
        print('%%%%%%%%%%%%%%%%')
        print('                  ')
        return target_root_copy


def extract_out_second_half(actual_xpath):#to be attachrd to path_base
    temp = actual_xpath.split('/')[-1]
    return temp
def handle_base_path(s):
    start = r_h[s][0]
#     print(f'start {start}')
    a= start
    specific_elem = a.split('/')[-1]
#     print(f'specific elem{specific_elem}')
    temp = a.split('/')[:-1]
#     print('temp',temp)
    temp_a = '/'.join(temp)
#     print('temp_a',temp_a)

    temp_b = temp[-1].split('[')[0]
    temp_c = temp
    temp_c[-1] = temp_b
    temp_d = '/'.join(temp_c)
#     print('temp_d',temp_d)
#     print('   ')
    return temp_d
            
def evaluate_if_got_multiple(current_test_case):
    occur_again = False
    for index,fields in enumerate(current_test_case):
        xpath_find = fields[0].strip()
        if index == 0:
            field_one = xpath_find
        if (xpath_find == field_one) and index != 0:
            occur_again = True
            return True
    return occur_again
def prepare_data_for_multiple(current_test_case):
    holding = []
    values_store=[]
    for index,fields in enumerate(current_test_case):
        
        path_to_store = fields[0]
        value = fields[1]
        values_store.append(value)
        if index == 0:
            temp = []
            
            field_one = path_to_store
            temp.append(path_to_store)
            
        elif (path_to_store == field_one) and (index != 0):
            holding.append(temp)
            temp = []

        
            temp.append(path_to_store)
        else:
#             print('  ----  ')
#             print('    ')
#             print(temp)
#             print('  ----  ')
#             print('    ')
            temp.append(path_to_store)
    holding.append(temp)
    return holding , values_store

def fix_space_typo(temp):
    temp_2 = temp.split("/")
    for index,i in enumerate(temp_2):
        temp_2[index] = i.strip()
    temp_3 = ' / '.join(temp_2)
    return temp_3

In [106]:
def remove_first_element(arr):
    if len(arr) > 0:
        t = arr.pop(0)
    return arr,t
def handle_multiple(current_test_case,test_case_id,r_h_original,target_root,desc):
    print(f' {test_case_id}  ')
    whole_count_pos = 0
    hold_1 , value_store = prepare_data_for_multiple(current_test_case)
    print('------------')
    print(hold_1)
    print('   ')
    field_1 = hold_1[0][0]
    base_path_field_1 = handle_base_path(field_1)

    print('   ')
    value_store_key = 0
    temp_base_path = base_path_field_1
    target_root_copy = copy.deepcopy(target_root)
    
    for index,each_case in enumerate(hold_1):
        
        r_h = r_h_original

        path_base = temp_base_path + f'[{index+1}]' + '/'
        

        for each_path in each_case:
            
            print('+++++++++++++++++++++++++')
            

            
            insert_value = str(value_store[value_store_key])
            print('+++++++++++++++++++++++++')
            print('---------------')
            print('   ')
            
            print(f'insert value = {value_store[value_store_key]}')
            if 'Field exists with missing value' in desc and value_store_key == 0:
                
                print(f'old insert value {insert_value} type is {type(insert_value)} id is {value_store_key}')
                print(f'##Description = {desc}##')
                insert_value = ''
                print(f'new insert value {insert_value}')
            value_store_key +=1
        
            print(f'value_store_key === > {value_store_key}')
            print(' ')
            base_path_each_path = handle_base_path(each_path)
            i=0
            if base_path_each_path == base_path_field_1:

                print('   ')
                print(f'----<<--Ammend base path for {each_path} ->>-----')
#                 if each_path == field_1 and whole_count_pos==0:
#                     print(']]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]')
#                     print(f'each_path {each_path} ')
#                     target_root_copy = delete_element_attrib(desc,target_root_copy,each_path,r_h)
#                 print('   ')
                if each_path in r_h:
                    r_h[each_path],actual_xpath = remove_first_element(r_h[each_path])
                    temp = extract_out_second_half(actual_xpath)
                    actual_xpath_absolute = path_base + temp
                    print(f'----<<--actual amended base path ==>{actual_xpath_absolute}')
                    target_element = target_root_copy.xpath(actual_xpath_absolute,namespaces = namespace_map)

                    print(f'----<<--actual path {each_path} index == {whole_count_pos}')
                    if '@' in each_path:
                        print(f'target element ==>{target_element}')
                        print(f'>>>>>>target_element.attrib = {target_element[0].attrib}')
                        print(f'>>>>>>target_element.text = {target_element[0].text}')
                        target_element = insert_for_attribute(target_element,insert_value,each_path)
                        print('             ')
                        
                        print('   ')
                    else:
                        target_element = insert_for_element(target_element,insert_value,each_path)
                    if whole_count_pos == 0:
                        print(']]]]]]]]]]]]]]]]]base_path_each_path == base_path_field_1]]]]]]]]]]]]]]]]]]')
                        print(f'each path = {each_path}')
                        print(f'{actual_xpath_absolute}')
                        target_root_copy = delete_element_attrib(desc,target_root_copy,actual_xpath_absolute,r_h,each_path)
                        print('<<<<last>>>')
      

            else:#base_path_each_path != base_path_field_1:
                    print('   ')
                    print(f'----<<--just change straight away {each_path} index == {whole_count_pos}->>')
                    print('   ')
                    if each_path in r_h:
                        r_h[each_path],actual_xpath = remove_first_element(r_h[each_path])
                    target_element = target_root_copy.xpath(actual_xpath,namespaces = namespace_map)
                    print('    ')
                    print('===============')
                    print(f'acutal path : {actual_xpath}')
                    
                    
                    print('   ')
                    if '@' in each_path:
                        print('   ')
                        print(f'target element {target_element}')
                        print('------change attrib------')
                        target_element = insert_for_attribute(target_element,insert_value,each_path)
                       
                        print('   ')
                    else:
                        print('   ')
                        print(f'target element {target_element}')
                        print('------change element------')
                        target_element = insert_for_element(target_element,insert_value,each_path)
                        

                        print('   ')
                    if whole_count_pos == 0:
                        print(']]]]]]]]]]]]]]]base_path_each_path != base_path_field_1]]]]]]]]]]]]]]]]]]]]')
                        print(f'each path = {each_path}')
                        print(f'{actual_xpath_absolute}')
                        target_root_copy = delete_element_attrib(desc,target_root_copy,actual_xpath,r_h)
                        print('<<<<last>>>')
#             if whole_count_pos == 0:

#                 print(f'{each_path}')
#                 print(f'{ea}')
#                 if '@' in each_path:
#                     pass
#                 print('<<<<last>>>')
            whole_count_pos += 1
  
    

    print('   ')
    print('             ')
    print(value_store_key)
    print('             ')
    print(value_store)
    return target_root_copy

In [118]:

KEY_NOT_FOUND=[]
elements_to_remove = []
error_prone_multiple = []
for i,test_case in enumerate(target_elements_syn):
        
    current_test_case = test_case['Fields']
    test_case_id = test_case['Test Case ID']

    print('############=================############')
    print(f'TEST CAES ID {test_case_id}')
    
    print('############=================############')

    desc = test_case['descript']
    sample_to_use = test_case['Sample']
    standard  = test_case['Standard']
    biz_pint = test_case['BIS/PINT']
    rule_id = test_case['Rule ID']
    
    
    f_p = file_paths_xml_files[sample_to_use]
    
    print(f'using {f_p}')
    print('   ')
    r_h = proper_mapping_final(f_p)
    
    if not r_h:
        print(f'cannot make r_h : {test_case_id}  sample :{sample_to_use}')
        continue
    tree = etree.parse(f_p)
    target_root = tree.getroot()
#     print(f'test case id :{test_case_id}')
#     print(desc)
    original_tree_copy = copy.deepcopy(target_root)
    original_tree_copy = adjust_identifier(original_tree_copy,filepath_base_identifier)
    if standard == 'PEPPOL':
        if biz_pint == 'BIS':
            value = 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0'
            adjust_customisation_id(original_tree_copy,value_cus)

        if biz_pint == 'PINT':
            value = 'urn:peppol:bis:billing'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:peppol:pint:billing-1@sg-1'
            adjust_customisation_id(original_tree_copy,value_cus)


    if standard == 'Non-PEPPOL':
        if biz_pint == 'BIS':
            value = 'urn:fdc:peppol.eu:2017:poacc:billing:01:1.0'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:LocalTaxInvoice:sg:1.0'
            adjust_customisation_id(original_tree_copy,value_cus)

        if biz_pint == 'PINT':
            value = 'urn:peppol:bis:billing'
            adjust_profile_id(original_tree_copy,value)
            value_cus = 'urn:peppol:pint:billing-1@sg-1:LocalTaxInvoice:sg:1.0'
            adjust_customisation_id(original_tree_copy,value_cus)
            
            
            
            
    
    lenght_of_fields = len(current_test_case)
    occur_again = evaluate_if_got_multiple(current_test_case)
    if not occur_again:
        print('CURRENT TEST CASE')
        print(current_test_case)
        for index,fields in enumerate(current_test_case):

            attrib_yes = False
            xpath_find = fields[0].strip()
            
            field_value = str(fields[1])
            print('++++++++++++++++++')
            print('++++++++bododododo++++++++++')
            if pd.isna(fields[1]) and 'does not exist' not in desc and 'missing value' not in desc:
                print(f'hulala back boi')
                field_value = 'NA'
                print(field_value)
       
            if '@' in xpath_find:
                    attrib_yes = True
            
            if 'does not exist' in desc and index == 0:
                if '@' in xpath_find:
                    print('why look at u youre an attribute')
                remove_element_flag = True
                print(f'xpath_find{xpath_find} |||||||')
                print('    ')
                temp = []
                if xpath_find in r_h:
                    xpath_target = r_h[xpath_find]
                else:
                    print(f'while trying to find xpath targtet to store in elements_to_remove,could not find key {xpath_find}')
                elements_to_remove_file_name = rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
                temp.append(elements_to_remove_file_name)
                temp.append(xpath_find)
                temp.append(xpath_target)
                temp.append(f_p)
                elements_to_remove.append(temp)
            if 'missing value' in desc and index == 0:
                print(f'old insert value {field_value} type is {type(field_value)}')
                print('      ')
                print(f'id == {index}')
                print(f'##Description = {desc}##')
                field_value = ''
                print(f'new insert value {field_value}')   
            if xpath_find in r_h:
                xpath_target = r_h[xpath_find][0]
                if original_tree_copy.xpath(xpath_target, namespaces=namespace_map):
                    element = original_tree_copy.xpath(xpath_target, namespaces=namespace_map)[0]
                    if not attrib_yes:
                        print(f'-non multiple-->insert test case id{test_case_id}  fieled value-->{field_value} xpath=={xpath_target}')    
                        element.text = field_value
                    else:
                        key_attrib = xpath_find.split('@')[-1]
                        print(f'test case id{test_case_id}  key attrib -->{key_attrib}')
                        element.attrib[key_attrib] = field_value
                        print('                                           ')
                if 'Field does not exist' in desc and index == 0:

                        print('Targeting element......')
                        if xpath_find in r_h:
                            xpath_target = r_h[xpath_find][0]
                            element_temp = original_tree_copy.xpath(xpath_target, namespaces=namespace_map)
                            if len(element_temp)>0:
                                element = element_temp[0]
                                if attrib_yes:
                                    del element.attrib[key_attrib]
                                else:

                                    parent = element.getparent()

                                    if element.getparent() is not None:
                                        element.getparent().remove(element)
                            else:
                                print(' ')
                                print(f'element to be removed {element} does not exist')
            else:#+++
                temp=[]
                temp.append(f'test case:{test_case_id} | uses {sample_to_use} key not found for {xpath_find}')
                print(f'--->test case:{test_case_id} | uses {sample_to_use} key not found for {xpath_find}')
                KEY_NOT_FOUND.append(temp)
        target_root = original_tree_copy
    else:
        error_prone_multiple.append(test_case_id)
        target_root = handle_multiple(current_test_case,test_case_id,r_h,original_tree_copy,desc)
    tree = etree.ElementTree(target_root)
    indexed_filename =rule_id +'_' + 'Test_case_' + str(test_case_id) + '_' + sheet_name + '.xml'
    filepath = os.path.join(folder_name, indexed_filename)
    print(f'writing {filepath}')
    tree.write(filepath, encoding='utf-8', xml_declaration=True)
             
       
        

############=================############
TEST CAES ID 1
############=================############
using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml
   
----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x15cb99740> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x15fb14940> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
CURRENT TEST CASE
[['ubl:Invoice / cac:AllowanceCharge / cbc:Amount', nan], ['ubl:Invoice / cac:AllowanceCharge / cbc:ChargeIndicator', 'false']]
++++++++++++++++++
++++++++bododododo++++++++++
xpath_findubl:Invoice / cac:AllowanceCharge / cbc:Amount |||||||
    
-non multiple-->insert test case id1  fieled value-->nan xpath==/*/*[2]/cac:AllowanceCharge/cbc:Amount
Targe

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x15fbe4080> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x15ff2aec0> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
CURRENT TEST CASE
[['ubl:Invoice / cac:AllowanceCharge / cbc:Amount', 345.987], ['ubl:Invoice / cac:AllowanceCharge / cbc:ChargeIndicator', 'true']]
++++++++++++++++++
++++++++bododododo++++++++++
-non multiple-->insert test case id44  fieled value-->345.987 xpath==/*/*[2]/cac:AllowanceCharge/cbc:Amount
++++++++++++++++++
++++++++bododododo++++++++++
-non multiple-->insert test case id44  fieled value-->true xpath==/*/*[2]/cac:AllowanceCharge/cbc:ChargeIndicator
writing /Users/cheongray/iras_api_make_xml/store_manual_fix_files/IRASC5-077_T

############=================############
TEST CAES ID 89
############=================############
using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_3.xml
   
----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x16848e180> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x15fe09b00> element text ->urn:peppol:pint:billing-1@sg-1:LocalTaxInvoice:sg:1.0
  
----
----
CURRENT TEST CASE
[['ubl:CreditNote / cac:AllowanceCharge / cbc:BaseAmount', nan], ['ubl:CreditNote / cac:AllowanceCharge / cbc:ChargeIndicator', 'false']]
++++++++++++++++++
++++++++bododododo++++++++++
xpath_findubl:CreditNote / cac:AllowanceCharge / cbc:BaseAmount |||||||
    
-non multiple-->insert test case id89  fieled value-->nan xpath==/*/*[2]/cac:AllowanceCharge/cbc:BaseAmount
Targeting element....

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x15ca9b500> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x16837ac80> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
CURRENT TEST CASE
[['ubl:Invoice / cac:TaxTotal / cbc:TaxAmount', 2567.84]]
++++++++++++++++++
++++++++bododododo++++++++++
-non multiple-->insert test case id136  fieled value-->2567.84 xpath==/*/*[2]/cac:TaxTotal/cbc:TaxAmount
writing /Users/cheongray/iras_api_make_xml/store_manual_fix_files/IRASC5-080_Test_case_136_4_Format Validation.xml
############=================############
TEST CAES ID 137
############=================############
using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml
   
----
--profile--
  
<Eleme

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x168127fc0> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x15caaa080> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
CURRENT TEST CASE
[['ubl:Invoice / cac:TaxTotal / cac:TaxSubtotal / cbc:TaxableAmount', 500000]]
++++++++++++++++++
++++++++bododododo++++++++++
-non multiple-->insert test case id180  fieled value-->500000 xpath==/*/*[2]/cac:TaxTotal/cac:TaxSubtotal/cbc:TaxableAmount
writing /Users/cheongray/iras_api_make_xml/store_manual_fix_files/IRASC5-082_Test_case_180_4_Format Validation.xml
############=================############
TEST CAES ID 181
############=================############
using /Users/cheongray/iras_api_clean_1/sample_xml_files/sam

...

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x15f88ac80> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x16810c680> element text ->urn:peppol:pint:billing-1@sg-1:LocalTaxInvoice:sg:1.0
  
----
----
CURRENT TEST CASE
[['ubl:CreditNote / cac:CreditNoteLine / cac:AllowanceCharge / cbc:BaseAmount', 6.985], ['ubl:CreditNote / cac:CreditNoteLine / cac:AllowanceCharge / cbc:ChargeIndicator', 'false']]
++++++++++++++++++
++++++++bododododo++++++++++
-non multiple-->insert test case id564  fieled value-->6.985 xpath==/*/*[2]/cac:CreditNoteLine/cac:AllowanceCharge/cbc:BaseAmount
++++++++++++++++++
++++++++bododododo++++++++++
-non multiple-->insert test case id564  fieled value-->false xpath==/*/*[2]/cac:CreditNoteLine/cac:AllowanceCharge/cbc:ChargeIndicator
writing /Users/cheongra

############=================############
TEST CAES ID 614
############=================############
using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_3.xml
   
----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x15bdb6a40> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x168406e40> element text ->urn:peppol:pint:billing-1@sg-1:LocalTaxInvoice:sg:1.0
  
----
----
CURRENT TEST CASE
[['ubl:CreditNote / cac:CreditNoteLine / cac:Price / cbc:PriceAmount', 980.25]]
++++++++++++++++++
++++++++bododododo++++++++++
-non multiple-->insert test case id614  fieled value-->980.25 xpath==/*/*[2]/cac:CreditNoteLine/cac:Price/cbc:PriceAmount
writing /Users/cheongray/iras_api_make_xml/store_manual_fix_files/IRASC5-097_Test_case_614_4_Format Validation.xml
############==============

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x15d6fa800> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x15d725980> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
CURRENT TEST CASE
[['ubl:Invoice / cac:InvoiceLine / cac:Price / cac:AllowanceCharge / cbc:BaseAmount', 689.44]]
++++++++++++++++++
++++++++bododododo++++++++++
-non multiple-->insert test case id663  fieled value-->689.44 xpath==/*/*[2]/cac:InvoiceLine/cac:Price/cac:AllowanceCharge/cbc:BaseAmount
writing /Users/cheongray/iras_api_make_xml/store_manual_fix_files/IRASC5-099_Test_case_663_4_Format Validation.xml
############=================############
TEST CAES ID 664
############=================############
using /Users/cheongray/iras_ap

----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x15dbdae80> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x15da5bb40> element text ->urn:cen.eu:en16931:2017#conformant#urn:fdc:peppol.eu:2017:poacc:billing:international:sg:3.0
  
----
----
CURRENT TEST CASE
[['ubl:Invoice / cac:AdditionalDocumentReference / cbc:DocumentDescription', 2000.011], ['ubl:Invoice / cac:AdditionalDocumentReference / cbc:DocumentTypeCode', 'sgdtotal-incl-gst'], ['ubl:Invoice / cac:AdditionalDocumentReference / cbc:ID', 'SGD']]
++++++++++++++++++
++++++++bododododo++++++++++
-non multiple-->insert test case id708  fieled value-->2000.011 xpath==/*/*[2]/cac:AdditionalDocumentReference[1]/cbc:DocumentDescription
++++++++++++++++++
++++++++bododododo++++++++++
-non multiple-->insert test case id708  fie

-non multiple-->insert test case id753  fieled value-->sgdtotal-excl-gst xpath==/*/*[2]/cac:AdditionalDocumentReference[1]/cbc:DocumentTypeCode
++++++++++++++++++
++++++++bododododo++++++++++
-non multiple-->insert test case id753  fieled value-->SGD xpath==/*/*[2]/cac:AdditionalDocumentReference[1]/cbc:ID
writing /Users/cheongray/iras_api_make_xml/store_manual_fix_files/IRASC5-101_Test_case_753_4_Format Validation.xml
############=================############
TEST CAES ID 754
############=================############
using /Users/cheongray/iras_api_clean_1/sample_xml_files/sample_13.xml
   
----
--profile--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}ProfileID at 0x15da97c00> element text ->urn:fdc:peppol.eu:2017:poacc:billing:01:1.0
  
----
----
----
--customisation--
  
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}CustomizationID at 0x15da96a40> element text ->urn:peppol:pint:billing-1@sg-1:LocalTaxInvoice:sg:1.

In [119]:
# get_mapping_for_xml_invoice('/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_2.xml')

In [120]:
elements_to_remove

[['IRASC5-076_Test_case_1_4_Format Validation.xml',
  'ubl:Invoice / cac:AllowanceCharge / cbc:Amount',
  ['/*/*[2]/cac:AllowanceCharge/cbc:Amount'],
  '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml'],
 ['IRASC5-076_Test_case_5_4_Format Validation.xml',
  'ubl:Invoice / cac:AllowanceCharge / cbc:Amount',
  ['/*/*[2]/cac:AllowanceCharge/cbc:Amount'],
  '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml'],
 ['IRASC5-076_Test_case_9_4_Format Validation.xml',
  'ubl:Invoice / cac:AllowanceCharge / cbc:Amount',
  ['/*/*[2]/cac:AllowanceCharge/cbc:Amount'],
  '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml'],
 ['IRASC5-076_Test_case_13_4_Format Validation.xml',
  'ubl:Invoice / cac:AllowanceCharge / cbc:Amount',
  ['/*/*[2]/cac:AllowanceCharge/cbc:Amount'],
  '/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_1.xml'],
 ['IRASC5-076_Test_case_17_4_Format Validation.xml',
  'ubl:CreditNote / cac:AllowanceCharge / cbc:Amount',
  ['/*/*[2]

In [121]:
import numpy as np
to_skip_for_manual_fix = []
def process_xml_files_in_folder_Invoice_ID(folder_path,elements_to_remove):
    check_how_many_removed = []
    remove_from_root = []
    xml_files = glob.glob(os.path.join(folder_path, '*.xml'))
    for index_file,xml_file in enumerate(xml_files):
        name = xml_file.split('/')[-1]
        for e in elements_to_remove:
            
            if name in e:
                file_name = e[0]
#                 xpath_target = e[-2]
                xpath_find = e[1]
                f_p = e[-1]
                tree_temp = etree.parse(f_p)
                r_h_original = proper_mapping_final(f_p)
                r_h = r_h_original
                if xpath_find in r_h:
                    r_h[xpath_find],xpath_target = remove_first_element(r_h[xpath_find])
                    target_root = tree_temp.getroot()
                    element = target_root.xpath(xpath_target,namespaces=namespace_map)[0]
                    print(f'file name {file_name}')
                    print(f'element text {element.text}')
                    if element.getparent() is not None:
                        print('++++++')
                        element.getparent().remove(element)
                        print(element)
                        check_how_many_removed.append(xml_file)
                        print('     ')

                    else:
                        remove_from_root.append(xpath_find)
                        target_root.remove(element)
                    tree = etree.ElementTree(target_root)
                    tree.write(xml_file, encoding='utf-8', xml_declaration=True)
                    continue
            else:
                continue
                
    return check_how_many_removed

check_num = process_xml_files_in_folder_Invoice_ID(folder_name,elements_to_remove)







file name IRASC5-095_Test_case_565_4_Format Validation.xml
element text 900.00
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}BaseAmount at 0x168338c80>
     
file name IRASC5-089_Test_case_389_4_Format Validation.xml
element text 0.00
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}PrepaidAmount at 0x15dbcec80>
     
file name IRASC5-093_Test_case_481_4_Format Validation.xml
element text 45.00
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}Amount at 0x1684423c0>
     
file name IRASC5-094_Test_case_525_4_Format Validation.xml
element text 45.00
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}Amount at 0x168339c80>
     
file name IRASC5-076_Test_case_17_4_Format Validation.xml
element text 105
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}Amount at 0x15dbce7c0>
     
file name IRASC5-088_Test_c

file name IRASC5-078_Test_case_77_4_Format Validation.xml
element text 0
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}BaseAmount at 0x15fcacac0>
     
file name IRASC5-095_Test_case_557_4_Format Validation.xml
element text 900.00
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}BaseAmount at 0x1683ab800>
     
file name IRASC5-089_Test_case_385_4_Format Validation.xml
element text 0.00
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}PrepaidAmount at 0x15db6c300>
     
file name IRASC5-079_Test_case_125_4_Format Validation.xml
element text 0
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}BaseAmount at 0x15fb21540>
     
file name IRASC5-093_Test_case_497_4_Format Validation.xml
element text 45.00
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}Amount at 0x15db4c840>
     
file name IRASC5-096_Test

file name IRASC5-099_Test_case_685_4_Format Validation.xml
element text 80
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}BaseAmount at 0x1681b0340>
     
file name IRASC5-095_Test_case_541_4_Format Validation.xml
element text 900.00
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}BaseAmount at 0x1683397c0>
     
file name IRASC5-079_Test_case_117_4_Format Validation.xml
element text 0
++++++
<Element {urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2}BaseAmount at 0x16836f400>
     


In [122]:
for i in check_num:
    a= i.split('Test_case')
    to_skip_for_manual_fix.append(int(a[1].split('_')[1]))
total_cases = [i for i in range(1,769)]
to_manual_fix = [item for item in total_cases if item not in to_skip_for_manual_fix]


array = np.array(to_manual_fix )
folder_name = '/Users/cheongray/iras_api_make_xml/store_manual_fix_files'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
new_file_path = os.path.join(folder_name, 'foramt_validation_manual_fix_files.txt')

# Save the array to a text file
np.savetxt(new_file_path, array, fmt='%d')

In [123]:
sample_9 = proper_mapping_final('/Users/cheongray/iras_api_clean_1/sample_xml_files/sample_9.xml')
sample_9

{'StandardBusinessDocument': ['/*'],
 'StandardBusinessDocument / StandardBusinessDocumentHeader': ['/*/*[1]'],
 'StandardBusinessDocument / StandardBusinessDocumentHeader / HeaderVersion': ['/*/*[1]/*[1]'],
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Sender': ['/*/*[1]/*[2]'],
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Sender / Identifier': '/*/*[1]/*[2]/*',
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Sender / Identifier / @Authority': ['/*/*[1]/*[2]/*'],
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Receiver': ['/*/*[1]/*[3]'],
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Receiver / Identifier': '/*/*[1]/*[3]/*',
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Receiver / Identifier / @Authority': ['/*/*[1]/*[3]/*'],
 'StandardBusinessDocument / StandardBusinessDocumentHeader / DocumentIdentification': ['/*/*[1]/*[4]'],
 'StandardBusinessDocument / StandardBusinessDocumentHeader / Doc